In [3]:
# --- prerequisites: pip install yfinance numpy pandas scipy

import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime

# Black-Scholes price & delta
def bs_price(S, K, T, r, sigma, option='call'):
    if T <= 0:
        return float(max(S - K, 0) if option=='call' else max(K - S, 0))
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option=='call':
        return float(S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2))
    else:
        return float(K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1))

def bs_delta(S, K, T, r, sigma, option='call'):
    if T <= 0:
        if option=='call': return float(1.0 if S>K else 0.0)
        else:             return float(-1.0 if S<K else 0.0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    return float(norm.cdf(d1) if option=='call' else norm.cdf(d1)-1)

# Parameters
ticker = "SPY"
strike = 450
expiry = "2025-06-20"
vol    = 0.15
r      = 0.01
start  = "2025-01-01"

# 1) Fetch underlying prices
serie = yf.download(ticker, start=start, end=expiry)["Close"].dropna()
dates = serie.index
prices = serie.values

# 2) Time to expiry in years (≈252 trading days)
exp_dt = datetime.fromisoformat(expiry)
T = np.array([(exp_dt - d).days/252 for d in dates])

# 3) BS price & delta each day
bs_prices = np.array([bs_price(S=prices[i], K=strike, T=T[i], r=r, sigma=vol)
                      for i in range(len(prices))])
deltas    = np.array([bs_delta(S=prices[i], K=strike, T=T[i], r=r, sigma=vol)
                      for i in range(len(prices))])

# 4) Daily P&L
delta_C = bs_prices[1:] - bs_prices[:-1]
delta_S = prices[1:]    - prices[:-1]
pnl_unhedged = delta_C
pnl_hedged   = delta_C - deltas[:-1]*delta_S

# 5) Totals & risk
total_unhedged = float(bs_prices[-1] - bs_prices[0])
total_hedged   = float(pnl_hedged.sum())
var_unhedged   = float(np.var(pnl_unhedged))
var_hedged     = float(np.var(pnl_hedged))

# 6) Print & verdict
print(f"Unhedged total P&L: {total_unhedged:.2f}")
print(f"Hedged   total P&L: {total_hedged:.2f}\n")

print(f"Unhedged P&L variance: {var_unhedged:.5f}")
print(f"Hedged   P&L variance: {var_hedged:.5f}\n")

if abs(total_hedged) < abs(total_unhedged):
    print("✔️ Hedging reduced net P&L magnitude.")
else:
    print("❌ Hedging increased net P&L magnitude.")

if var_hedged < var_unhedged:
    print("✔️ Hedging reduced daily P&L variance (lower risk).")
else:
    print("❌ Hedging increased daily P&L variance.")


[*********************100%***********************]  1 of 1 completed

Unhedged total P&L: -34.58
Hedged   total P&L: -202.32

Unhedged P&L variance: 100.13310
Hedged   P&L variance: 202.67938

❌ Hedging increased net P&L magnitude.
❌ Hedging increased daily P&L variance.



/var/folders/_5/g20j99hd5j91r849_d_zgb040000gn/T/ipykernel_81042/3408225046.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2))
/var/folders/_5/g20j99hd5j91r849_d_zgb040000gn/T/ipykernel_81042/3408225046.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(norm.cdf(d1) if option=='call' else norm.cdf(d1)-1)
